In [1]:
using BenchmarkTools

In [2]:
W = trues(30000, 30001)
W[rand(1:length(W), round(Int, √length(W)))] .= 0
W

30000×30001 BitMatrix:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1   

In [3]:
function get_S(W)
    zeroidxs = getindex.(findall(iszero, W), [1 2])
    S1 = Set(zeroidxs[:,1])
    S2 = Set(zeroidxs[:,2])
    return S1, S2
end

get_S (generic function with 1 method)

In [4]:
@btime get_S($W)

  1.495 s (37 allocations: 3.04 MiB)


(Set([29965, 11950, 12427, 7685, 18374, 3406, 28900, 27640, 28576, 2015  …  12084, 1711, 7108, 15380, 15271, 17984, 4496, 8546, 8992, 4159]), Set([29965, 12427, 18374, 3406, 28804, 27640, 28576, 1090, 18139, 28165  …  4233, 21639, 21616, 15271, 17984, 4496, 8546, 4159, 8992, 24824]))

In [5]:
function get_S1(W)
    zeroidxs = getindex.(findall(iszero, W), [1 2])
    S1 = unique(zeroidxs[:,1])
    S2 = unique(zeroidxs[:,2])
    return S1, S2
end

get_S1 (generic function with 1 method)

In [6]:
@btime get_S1($W)

  1.499 s (107 allocations: 4.42 MiB)


([8289, 7107, 12523, 22616, 12695, 13270, 22921, 4059, 10791, 20055  …  28280, 13812, 14076, 18495, 3177, 6939, 25295, 29014, 1382, 1176], [2, 4, 5, 7, 10, 11, 13, 14, 15, 16  …  29991, 29992, 29993, 29994, 29995, 29996, 29998, 29999, 30000, 30001])

In [7]:
sort.(unique.(collect.(get_S(W)))) == sort.(get_S1(W))

true

In [8]:
function get_S2(W)
    m, n = size(W)
    b1, b2 = fill(false, m), fill(false, n)
    Threads.@threads for j in 1:n
        for i in 1:m
            @inbounds if W[i, j] == 0
                b1[i] = b2[j] = true
            end
        end
    end
    b1, b2
end

get_S2 (generic function with 1 method)

In [9]:
@btime get_S2($W)

  79.599 ms (65 allocations: 64.98 KiB)


(Bool[1, 1, 0, 1, 1, 1, 0, 0, 1, 0  …  1, 0, 1, 1, 0, 0, 0, 0, 1, 1], Bool[0, 1, 0, 1, 1, 0, 1, 0, 0, 1  …  1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [10]:
sort.(get_S1(W)) == get_S2(W)

false

In [11]:
get_S(W) |> typeof

Tuple{Set{Int64}, Set{Int64}}

In [12]:
get_S1(W) |> typeof

Tuple{Vector{Int64}, Vector{Int64}}

In [13]:
get_S2(W) |> typeof

Tuple{Vector{Bool}, Vector{Bool}}